In [1]:
%load_ext sparkmagic.magics

In [22]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

HttpClientException: Invalid status code '404' from http://10.34.14.129:8999/sessions/0 with error payload: "Session '0' not found."

Cleaned up endpoint http://10.34.14.129:8999
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
74,application_1551772643782_0017,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [23]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType, IntegerType
import pyspark.sql.functions as F

In [24]:
schema_acq = [['LoanID', str],
              ['Channel', str],
              ['SellerName', str],
              ['OrInterestRate', float],
              ['OrUnpaidPrinc', int],
              ['OrLoanTerm', int],
              ['OrDate', str],
              ['FirstPayment', str],
              ['OrLTV', float],
              ['OrCLTV', float],
              ['NumBorrow', float],
              ['DTIRat', float],
              ['CreditScore', float],
              ['FTHomeBuyer', str],
              ['LoanPurpose', str],
              ['PropertyType', str],
              ['NumUnits', int],
              ['OccStatus', str],
              ['PropertyState', str],
              ['Zip', int],
              ['MortInsPerc', float],
              ['ProductType', str],
              ['CoCreditScore', float],
              ['MortInsType', float],
              ['RelMortInd', str]]

schema_per = [['LoanID', str],
              ['ReportingDate', str],
              ['Servicer', str],
              ['CurrInterestRate', float],
              ['CAUPB', float],
              ['LoanAge', int],
              ['MonthsToMaturity', float],
              ['AdMonthsToMaturity', float],
              ['MaturityDate', str],
              ['MSA', int],
              ['CurDelStatus', str],
              ['ModFlag', str],
              ['ZeroBalCode', float],
              ['ZeroBalEffDate', str],
              ['LastInstallDate', str],
              ['ForeclosureDate', str],
              ['DispositionDate', str],
              ['ForeclosureCost', float],
              ['RepairCost', float],
              ['AssetRecCost', float],
              ['MiscCostsPF', float],
              ['ATFHP', float],
              ['NetSaleProceeds', float],
              ['CreditEnhProceeds', float],
              ['RPMWP', float],
              ['OtherForePro', float],
              ['NonInterestUPB', float],
              ['PricipleForgiven', float],
              ['RMWPF', str],
              ['FPWA', float],
              ['ServicingIndicator', str]]


In [25]:
schemap = {
    str: StringType(),
    float: FloatType(),
    int: IntegerType()
    
}

In [26]:
schema_acq_spark = StructType([StructField(k, schemap[v], True) for k,v in schema_acq])
schema_per_spark = StructType([StructField(k, schemap[v], True) for k,v in schema_per])

acq = spark.read.load("/Fannie-Mae/2017/Acquisition/", format="csv", header="false",
                     sep='|', schema=schema_acq_spark)

per = spark.read.load("/Fannie-Mae/2017/Performance/", format="csv", header="false",
                     sep='|', schema=schema_per_spark)


In [27]:
per = per.withColumn('date', F.to_date(per.ReportingDate, 'MM/dd/yyyy')) \
    .drop('ReportingDate') \
    .withColumnRenamed('date', 'ReportingDate') \
    .orderBy("LoanID", F.desc("ReportingDate")) \
    .dropDuplicates(["LoanID"])

df = acq.join(per, 'LoanID', 'outer')
df.write.format('com.databricks.spark.csv').save('/Fannie-Mae/2017/Output.csv')

print(df.count())



1949478